# Boilerplate

In [1]:
# python internal 
import collections
import copy
import gc
from glob import glob
import h5py
import itertools
import os
print(os.getcwd())
import random
import re
import socket
print(socket.gethostname())
import shutil
import subprocess
import sys
# conda/pip
import dask
import graphviz
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy
import seaborn as sns
import tensorflow as tf
from tqdm import tqdm
# special packages on the DIGS
import py3Dmol
import pymol
import pyrosetta
# notebook magic
%matplotlib inline
%load_ext autoreload
%autoreload 2

/mnt/home/pleung/projects/bistable_bundle/r3/hinges
dig30


# Flo's original approach:
4. now it gets even worse. I use`/home/flop/switch/5thround/DHRs/loops/score_to_loop.ipynb` to generate cmds for loop closure. have a look, and then we should probably talk about it. it is messy.
	
basically I first try to find loops that match the original DHR, for example
in: `/home/flop/switch/5thround/DHRs/loops/match_orig_d6_04/`

because this doesn't work for all of them, I also re-loop the original DHRs,
also sampling truncations and extensions of the helices before and after the
new loop

i exhaustively did that for all dhrs in my input set: 
`/home/flop/switch/5thround/DHRs/loops/x_ind_all_te`

then I try to find loops in the new states that match the outputs of the state
x relooping: `/home/flop/switch/5thround/DHRs/loops/match_ind_d6/`

# I will follow Flo's looping procedure with some changes.
I will use the serialization build of PyRosetta to enable recording user defined info about the designs.  
This enables downstream inline filtering and data analysis, as well as clustering by lineage.
I will try the following hierarchy of closure methods: 
1. close using identical length loop and no adjustment with connect chains mover (CCM) at 0.8 RMS (total length and SS match)
2. close using identical length loop and +/-2 res adjustment on each side with CCM at 0.4 RMS (total length match, potential SS mismatch)
3. constrained remodel with `BluePrintBDR` (total length match, potential SS mismatch)

# Make functions for looping and labeling to assist downstream penultimate design step

In [2]:
from pyrosetta.distributed.packed_pose.core import PackedPose
from pyrosetta.distributed import requires_init
from typing import *

@requires_init
def load(silent: str, **kwargs) -> Generator[str, PackedPose, None]:
    """
    Requires init. Because of some issues with silent energies, if your energy 
    is supposed to be a string but is an empty string  (""),
    it will be set to a float value of 0.0
    @pleung
    """
    import pyrosetta
    import pyrosetta.distributed.io as io
    from pyrosetta.rosetta.core.io.silent import SilentFileOptions, SilentFileData
    from pyrosetta.rosetta.core.pose import Pose
    if silent == None:
        silent = kwargs["-in:file:silent"]
    else:
        pass
    options = SilentFileOptions() 
    sfd = SilentFileData(options) # the part that requires init
    sfd.read_file(silent)
    for tag in sfd.tags():
        ss = sfd.get_structure(tag)
        ss_energies = list(ss.energies())
        pose = Pose()
        ss.fill_pose(pose)
        pose.pdb_info().name(tag)
        for energy in ss_energies:
            key = energy.name() 
            # takes advantage of string_value returning "" for true floats
            if energy.string_value() is not "":
                value = energy.string_value() 
            else: # if your energy is supposed to be an empty string it will be set to 0.0 here
                value = energy.value()
            pyrosetta.rosetta.core.pose.setPoseExtraScore(pose, key, value)
        ppose = io.to_packed(pose)
        yield ppose
        
def loop_match(ppose: PackedPose, **kwargs) -> PackedPose:
    """
    Match loop length, total length and DSSP with parent. Strictest method of closure.
    """
    from copy import deepcopy
    import pyrosetta
    import pyrosetta.distributed.io as io
    from pyrosetta.distributed.tasks.rosetta_scripts import SingleoutputRosettaScriptsTask
    scores = deepcopy(ppose.pose.scores)
    # get parent from ppose, get loop length from parent length - ppose length
    parent_length = int(scores["parent_length"])
    length = int(parent_length - len(ppose.pose.residues))
    xml = """
    <ROSETTASCRIPTS>
        <SCOREFXNS>
            <ScoreFunction name="sfxn" weights="beta_nov16" /> 
        </SCOREFXNS>
        <RESIDUE_SELECTORS>          
        </RESIDUE_SELECTORS>
        <TASKOPERATIONS>
        </TASKOPERATIONS>
        <SIMPLE_METRICS>
        </SIMPLE_METRICS>
        <MOVERS>
            <ConnectChainsMover name="closer" 
                chain_connections="[A+B]" 
                loopLengthRange="{length},{length}" 
                resAdjustmentRangeSide1="0,0" 
                resAdjustmentRangeSide2="0,0" 
                RMSthreshold="0.8"/>
            <SwitchChainOrder name="rechain" chain_order="1"/>
        </MOVERS>
        <FILTERS>
        </FILTERS>
        <PROTOCOLS>
            <Add mover_name="closer"/>
            <Add mover_name="rechain"/>
        </PROTOCOLS>
    </ROSETTASCRIPTS>
    """.format(length=length)
    closer = SingleoutputRosettaScriptsTask(xml)
    try:
        maybe_closed_ppose = closer(ppose.pose.clone())
        closure_type = "loop_match"
    except RuntimeError:
        maybe_closed_ppose = io.to_pose(ppose.pose.clone())
        closure_type = "not_closed"
    maybe_closed_pose = io.to_pose(maybe_closed_ppose)
    pyrosetta.rosetta.core.pose.setPoseExtraScore(maybe_closed_pose, "closure_type", closure_type)
    for key, value in scores.items():
        pyrosetta.rosetta.core.pose.setPoseExtraScore(maybe_closed_pose, key, value)
    final_ppose = io.to_packed(maybe_closed_pose)
    return final_ppose

def length_match(ppose: PackedPose, **kwargs) -> list:
    """
    Match total length with parent. DSSP can be off by 1 on both sides of the loop. Less strict method of closure.
    TODO bugs with length matching :( I think it is due to TJ mover not accepting longer than 5
    TODO one way to get around bugs is to return a filtered list of closed poses that are the correct length...
    """
    from copy import deepcopy
    import pyrosetta
    import pyrosetta.distributed.io as io
    from pyrosetta.distributed.tasks.rosetta_scripts import SingleoutputRosettaScriptsTask
    # ensure pose still needs to be closed
    if ppose.pose.num_chains() == 1:
        final_pposes = [io.to_packed(ppose.pose.clone())]
        return final_pposes
    else:
        scores = deepcopy(ppose.pose.scores)
        # get parent from ppose, get loop length from parent length - ppose length
        parent_length = int(scores["parent_length"])
        length = int(parent_length - len(ppose.pose.residues))
        combinations = [
            {"adj1" : -1, "adj2" :  1, "length" : length},
            {"adj1" :  1, "adj2" : -1, "length" : length},
            {"adj1" :  0, "adj2" : -1, "length" : length+1},
            {"adj1" : -1, "adj2" :  0, "length" : length+1},
            {"adj1" :  0, "adj2" :  1, "length" : length-1},
            {"adj1" :  1, "adj2" :  0, "length" : length-1},
            {"adj1" : -1, "adj2" : -1, "length" : length+2},
            {"adj1" :  1, "adj2" :  1, "length" : length-2},
            {"adj1" : -2, "adj2" :  2, "length" : length},
            {"adj1" :  2, "adj2" : -2, "length" : length},
            {"adj1" : -2, "adj2" :  1, "length" : length+1},
            {"adj1" :  1, "adj2" : -2, "length" : length+1},
            {"adj1" :  2, "adj2" : -1, "length" : length-1},
            {"adj1" : -1, "adj2" :  2, "length" : length-1},
            {"adj1" :  0, "adj2" : -2, "length" : length+2},
            {"adj1" : -2, "adj2" :  0, "length" : length+2},
            {"adj1" :  0, "adj2" :  2, "length" : length-2},
            {"adj1" :  2, "adj2" :  0, "length" : length-2},
        ]
        closed_pposes = []
        for combo in combinations:
            if combo["length"] > 5: # mover can't accomodate loops longer than 5
                continue
            elif combo["length"] < 1: # mover can't accomodate loops shorter than 1
                continue
            elif int(sum(combo.values())) != length: # really just a bug check
                continue
            else:
                xml = """
                <ROSETTASCRIPTS>
                    <SCOREFXNS>
                        <ScoreFunction name="sfxn" weights="beta_nov16" /> 
                    </SCOREFXNS>
                    <RESIDUE_SELECTORS>          
                    </RESIDUE_SELECTORS>
                    <TASKOPERATIONS>
                    </TASKOPERATIONS>
                    <SIMPLE_METRICS>
                    </SIMPLE_METRICS>
                    <MOVERS>
                        <ConnectChainsMover name="closer" 
                            chain_connections="[A+B]" 
                            loopLengthRange="{length},{length}" 
                            resAdjustmentRangeSide1="{adj1},{adj1}" 
                            resAdjustmentRangeSide2="{adj2},{adj2}" 
                            RMSthreshold="0.8"/>
                        <SwitchChainOrder name="rechain" chain_order="1"/>
                    </MOVERS>
                    <FILTERS>
                    </FILTERS>
                    <PROTOCOLS>
                        <Add mover_name="closer"/>
                        <Add mover_name="rechain"/>
                    </PROTOCOLS>
                </ROSETTASCRIPTS>
                """.format(**combo)
                closer = SingleoutputRosettaScriptsTask(xml)
                try: # try to close the pose with restrictive param set
                    print("Closing with the following params:", combo)
                    maybe_closed_ppose = closer(ppose.pose.clone())
                    print("Closed with the following params:", combo)
                    # check that the length is right
                    if len(maybe_closed_ppose.pose.residues) == parent_length:
                        closed_pposes.append(maybe_closed_ppose)
                    else:
                        pass
                except RuntimeError: # if the pose isn't closed, go to next iteration
                    print("Failed with the following params:", combo)
    # check if any poses were closed and had the right length, if so just return the original input
    if len(closed_pposes) == 0:
        maybe_closed_poses = [ppose.pose.clone()]
    # otherwise at least one had the right length
    else:
        maybe_closed_poses = [io.to_pose(ppose) for ppose in closed_pposes]
        # set scores
        for maybe_closed_pose in maybe_closed_poses:
            for key, value in scores.items():
                pyrosetta.rosetta.core.pose.setPoseExtraScore(maybe_closed_pose, key, value)
            pyrosetta.rosetta.core.pose.setPoseExtraScore(maybe_closed_pose, "closure_type", "length_match")
    # pack results
    final_pposes = [io.to_packed(maybe_closed_pose) for maybe_closed_pose in maybe_closed_poses]
    return final_pposes

def strict_remodel(ppose: PackedPose, **kwargs) -> PackedPose:
    """
    DSSP and SS agnostic in principle but in practice more or less matches.
    """
    from copy import deepcopy
    import os
    import pyrosetta
    import pyrosetta.distributed.io as io
    from pyrosetta.distributed.packed_pose.core import PackedPose
    from pyrosetta.distributed.tasks.rosetta_scripts import SingleoutputRosettaScriptsTask
    
    def strict_remodel_helper(ppose: PackedPose, loop_length:int) -> str:
        import binascii, os
        import pyrosetta
        from pyrosetta.rosetta.core.pose import Pose
        
        def phi_psi_omega_to_abego(phi: float, psi: float, omega: float) -> str:
            """
            From Buwei
            https://wiki.ipd.uw.edu/protocols/dry_lab/rosetta/scaffold_generation_with_piecewise_blueprint_builder
            """
            if psi == None or phi == None: return "X"
            if omega == None: omega = 180

            if abs(omega) < 90:
                return "O"
            elif phi > 0:
                if -100.0 <= psi < 100:
                    return "G"
                else:
                    return "E"
            else:
                if -75.0 <= psi < 50:
                    return "A"
                else:
                    return "B"
            return "X"

        def abego_string(phi_psi_omega: list) -> str:
            """
            From Buwei
            https://wiki.ipd.uw.edu/protocols/dry_lab/rosetta/scaffold_generation_with_piecewise_blueprint_builder
            """
            out = ""
            for x in phi_psi_omega:
                out += phi_psi_omega_to_abego(x[0], x[1], x[2])
            return out

        def get_torsions(pose: Pose) -> list:
            """
            From Buwei
            https://wiki.ipd.uw.edu/protocols/dry_lab/rosetta/scaffold_generation_with_piecewise_blueprint_builder
            """
            torsions=[]
            for i in range(1, pose.total_residue() + 1):
                phi = pose.phi(i)
                psi = pose.psi(i)
                omega = pose.omega(i)
                if i == 1:
                    phi = None
                if i == pose.total_residue():
                    psi = None
                    omega = None
                torsions.append((phi, psi, omega))
            return torsions
        
        tors = get_torsions(ppose.pose)
        abego_str = abego_string(tors)
        dssp = pyrosetta.rosetta.protocols.simple_filters.dssp(ppose.pose)
        # name blueprint a random 32 long hex string
        filename = str(binascii.b2a_hex(os.urandom(16)).decode("utf-8")) + ".bp"
        # write a temporary blueprint file
        with open(filename, "w+") as f:
            end1, begin2 = ppose.pose.chain_end(1), ppose.pose.chain_begin(2)
            end2 = ppose.pose.chain_end(2)
            for i in range(1, end1+1):
                if i == end1: 
                    print(
                        str(i), 
                        ppose.pose.residue(i).name1(), 
                        dssp[i-1]+"X",
                        "R",
                        file=f,
                    )
                else:
                    print(
                        str(i),
                        ppose.pose.residue(i).name1(), 
                        dssp[i-1]+abego_str[i-1], 
                        ".", 
                        file=f,
                    )
            for i in range(loop_length):
                print("0", "V", "DX", "R", file=f)
            for i in range(begin2, end2+1):
                if i == begin2: 
                    print(
                        str(i), 
                        ppose.pose.residue(i).name1(), 
                        dssp[i-1]+"X",
                        "R",
                        file=f,
                    )
                else:
                    print(
                        str(i),
                        ppose.pose.residue(i).name1(), 
                        dssp[i-1]+abego_str[i-1], 
                        ".", 
                        file=f,
                    )
        return filename
    
    # ensure pose still needs to be closed
    if ppose.pose.num_chains() == 1:
        final_ppose = io.to_packed(ppose.pose.clone())
        return final_ppose
    else:
        scores = deepcopy(ppose.pose.scores)
        # get parent from ppose, get loop length from parent length - ppose length
        parent_length = int(scores["parent_length"])
        length = int(parent_length - len(ppose.pose.residues))
        bp = strict_remodel_helper(ppose, length)
        xml = """
        <ROSETTASCRIPTS>
            <SCOREFXNS>
                <ScoreFunction name="sfxn1" weights="fldsgn_cen">
                    <Reweight scoretype="hbond_sr_bb" weight="1.0" />
                    <Reweight scoretype="hbond_lr_bb" weight="1.0" />
                    <Reweight scoretype="atom_pair_constraint" weight="1.0" />
                    <Reweight scoretype="angle_constraint" weight="1.0" />
                    <Reweight scoretype="dihedral_constraint" weight="1.0" />
                </ScoreFunction>
            </SCOREFXNS>
            <RESIDUE_SELECTORS>          
            </RESIDUE_SELECTORS>
            <TASKOPERATIONS>
            </TASKOPERATIONS>
            <SIMPLE_METRICS>
            </SIMPLE_METRICS>
            <MOVERS>
                <BluePrintBDR name="bdr" 
                blueprint="{bp}" 
                use_abego_bias="0" 
                use_sequence_bias="0" 
                rmdl_attempts="20"
                scorefxn="sfxn1"/>
                #RemodelMover name="bdr" blueprint="{bp}"/>      
            </MOVERS>
            <FILTERS>
            </FILTERS>
            <PROTOCOLS>
                <Add mover_name="bdr"/>
            </PROTOCOLS>
        </ROSETTASCRIPTS>
        """.format(bp=bp)
        strict_remodel = SingleoutputRosettaScriptsTask(xml)
        # try to close once
        maybe_closed_ppose = strict_remodel(ppose.pose.clone())
        for i in range(5):
            if maybe_closed_ppose is not None: # check if it worked
                closure_type = "strict_remodel"
                break
            else: # try again if it didn't
                print(f"attempt: {i}")
                maybe_closed_ppose = strict_remodel(ppose.pose.clone())
        os.remove(bp) # cleanup tree
        if maybe_closed_ppose is not None:
            closure_type = "strict_remodel" 
            maybe_closed_pose = io.to_pose(maybe_closed_ppose)
            pyrosetta.rosetta.core.pose.setPoseExtraScore(maybe_closed_pose, "closure_type", closure_type)
            final_ppose = io.to_packed(maybe_closed_pose)
            return final_ppose
        else:
            final_ppose = io.to_packed(ppose.pose.clone())
            return final_ppose
    
    
def label(ppose: PackedPose, **kwargs) -> PackedPose:
    """
    TODO this seems to have the expected behavior.
    """
    from copy import deepcopy
    import pyrosetta
    import pyrosetta.distributed.io as io
    from pyrosetta.distributed.tasks.rosetta_scripts import SingleoutputRosettaScriptsTask
    from pyrosetta.rosetta.core.pose import Pose
    from pyrosetta.rosetta.core.select import get_residues_from_subset
    from pyrosetta.rosetta.protocols.rosetta_scripts import XmlObjects 
    
    def phi_psi_omega_to_abego(phi: float, psi: float, omega: float) -> str:
        """
        From Buwei
        https://wiki.ipd.uw.edu/protocols/dry_lab/rosetta/scaffold_generation_with_piecewise_blueprint_builder
        """
        if psi == None or phi == None: return "X"
        if omega == None: omega = 180

        if abs(omega) < 90:
            return "O"
        elif phi > 0:
            if -100.0 <= psi < 100:
                return "G"
            else:
                return "E"
        else:
            if -75.0 <= psi < 50:
                return "A"
            else:
                return "B"
        return "X"
    
    def abego_string(phi_psi_omega: list) -> str:
            """
            From Buwei
            https://wiki.ipd.uw.edu/protocols/dry_lab/rosetta/scaffold_generation_with_piecewise_blueprint_builder
            """
            out = ""
            for x in phi_psi_omega:
                out += phi_psi_omega_to_abego(x[0], x[1], x[2])
            return out

    def get_torsions(pose: Pose) -> list:
        """
        From Buwei
        https://wiki.ipd.uw.edu/protocols/dry_lab/rosetta/scaffold_generation_with_piecewise_blueprint_builder
        """
        torsions=[]
        for i in range(1, pose.total_residue() + 1):
            phi = pose.phi(i)
            psi = pose.psi(i)
            omega = pose.omega(i)
            if i == 1:
                phi = None
            if i == pose.total_residue():
                psi = None
                omega = None
            torsions.append((phi, psi, omega))
        return torsions
        
    def find_vv(seq):
        indices = []
        seq_minus_one = seq[:-1]
        for i, char in enumerate(seq_minus_one):
            if (char == seq[i+1]) and (char == "V"):
                indices.append(i+1)
            else:
                pass
        # rosetta sequence indexes begin at 1
        true_indices = [str(x+1) for x in indices]
        return true_indices
    
    scores = deepcopy(ppose.pose.scores)
    if ppose.pose.num_chains() != 1: # ensure pose has been closed, 
        new_loop_str = "0,0"
        labeled_pose = ppose.pose.clone()
    else:
        seq = str(ppose.pose.sequence())
        vv_indices = ",".join(find_vv(seq))
        pivot_helix = int(scores["pivot_helix"])
        pre_break_helix = int(scores["pre_break_helix"])
        # get helix indices for the pre and post break helices
        lower = pre_break_helix
        upper = pre_break_helix + 1
        xml = """
        <ROSETTASCRIPTS>
            <SCOREFXNS>
                <ScoreFunction name="sfxn" weights="beta_nov16" /> 
            </SCOREFXNS>
            <RESIDUE_SELECTORS>
                <SSElement name="middle" selection="{lower},H,E" to_selection="{upper},H,S" chain="A" reassign_short_terminal_loop="2" />
                <Index name="polyval_all" resnums="{vv_indices}" />
                <And name="polyval" selectors="middle,polyval_all" />
                <PrimarySequenceNeighborhood name="entire_val" selector="polyval" lower="5" upper="5" />
                <SecondaryStructure name="loop" overlap="0" minH="3" minE="2" include_terminal_loops="true" use_dssp="true" ss="L"/>
                <And name="new_loop_center" selectors="entire_val,loop" />
                <PrimarySequenceNeighborhood name="entire_new_loop_broad" selector="new_loop_center" lower="5" upper="5" />
                <ResidueName name="isval" residue_name3="VAL" />
                <And name="entire_new_loop" selectors="entire_new_loop_broad,isval" />
            </RESIDUE_SELECTORS>
            <TASKOPERATIONS>
            </TASKOPERATIONS>
            <SIMPLE_METRICS>
            </SIMPLE_METRICS>
            <MOVERS>
                <AddResidueLabel name="add_loop_label" residue_selector="entire_new_loop" label="new_loop" />  
            </MOVERS>
            <FILTERS>
            </FILTERS>
            <PROTOCOLS>
                <Add mover="add_loop_label" />
            </PROTOCOLS>
            #OUTPUT scorefxn="sfxn" />
        </ROSETTASCRIPTS>
        """.format(lower=lower, upper=upper, vv_indices=vv_indices)
        labeled = SingleoutputRosettaScriptsTask(xml)
        xml_obj = XmlObjects.create_from_string(xml)
        entire_new_loop_sel = xml_obj.get_residue_selector("entire_new_loop")
        labeled_ppose = labeled(ppose.pose.clone())
        labeled_pose = io.to_pose(labeled_ppose)
        new_loop_resis = list(get_residues_from_subset(entire_new_loop_sel.apply(labeled_pose)))
        new_loop_str = ",".join(str(resi) for resi in new_loop_resis)
    pyrosetta.rosetta.core.pose.setPoseExtraScore(labeled_pose, "new_loop_resis", new_loop_str)
    total_length = len(labeled_pose.residues)
    pyrosetta.rosetta.core.pose.setPoseExtraScore(labeled_pose, "total_length", total_length)
    dssp = pyrosetta.rosetta.protocols.simple_filters.dssp(labeled_pose)
    pyrosetta.rosetta.core.pose.setPoseExtraScore(labeled_pose, "dssp", dssp)
    tors = get_torsions(labeled_pose)
    abego_str = abego_string(tors)
    pyrosetta.rosetta.core.pose.setPoseExtraScore(labeled_pose, "abego_str", abego_str)
    for key, value in scores.items():
        pyrosetta.rosetta.core.pose.setPoseExtraScore(labeled_pose, key, value)
    final_ppose = io.to_packed(labeled_pose)
    return final_ppose

In [3]:
# def remodel_helper(ppose: PackedPose, loop_length:int) -> str:
#     import binascii, os  
#     # name blueprint a random 32 long hex string
#     filename = str(binascii.b2a_hex(os.urandom(16)).decode("utf-8")) + ".bp"
#     dssp = pyrosetta.rosetta.protocols.simple_filters.dssp(ppose.pose)
#     with open(filename, "w+") as f:
#         end1, begin2 = ppose.pose.chain_end(1), ppose.pose.chain_begin(2)
#         end2 = ppose.pose.chain_end(2)
#         for i in range(1, end1+1):
#             if i == end1: 
#                 print(
#                     str(i), 
#                     ppose.pose.residue(i).name1(), 
#                     dssp[i-1], 
#                     file=f
#                 )
#             else:
#                 print(str(i), ppose.pose.residue(i).name1(), ".", file=f)
#         for i in range(loop_length):
#             print("0", "x", "L", "PIKAA", "V", file=f)
#         for i in range(begin2, end2+1):
#             if i == begin2: 
#                 print(
#                     str(i),
#                     ppose.pose.residue(i).name1(), 
#                     dssp[i-1],
#                     file=f
#                 )
#             else:
#                 print(str(i), ppose.pose.residue(i).name1(), ".", file=f)
#     return filename

# bp = remodel_helper(tppose, 3)

In [3]:
# import pyrosetta.distributed.io as io
# pyrosetta.init(
#     " ".join(
#         [
#             "-corrections::beta_nov16",
#             "-holes:dalphaball /home/bcov/ppi/tutorial_build/main/source/external/DAlpahBall/DAlphaBall.gcc",
#             "-indexed_structure_store:fragment_store /net/databases/VALL_clustered/connect_chains/ss_grouped_vall_helix_shortLoop.h5",
#             "-mute all",
#         ] # TODO set seed 1
#     )
# )
# for i, tppose in enumerate(load("02_silents/states.silent")):
#     if i > 30:
#         break
#     else:
#         test_bdr = loop_match(tppose)
#         test_bdr = strict_remodel(test_bdr)
#         l = label(test_bdr)
#         l.pose.dump_pdb(f"t{i}.pdb")

# Setup Dask
Trying a adaptive SLURMCluster. to see the dashboard, forward port `8787` to `8000`:  
`local$ ssh -L 8000:localhost:8787 $USER@$HOSTNAME`  
now, the web UI is visible at `localhost:8000`  
if you're using a local cluster make sure the node this notebook is on has the same 
number of workers as cores

In [4]:
!echo $HOSTNAME
!echo $USER

dig30
pleung


In [6]:
from dask.distributed import Client
from dask_jobqueue import SLURMCluster

cluster = SLURMCluster(cores=1,
                       processes=1,
                       job_cpu=1,
                       memory="4GB",
                       queue="medium",
                       # TODO check if these two are neccessary
                       walltime="23:30:00",
                       death_timeout=600,
                      )
print(cluster.job_script())
# scale between 0 and 360 workers as needed
cluster.adapt(minimum=0, maximum=360) 
client = Client(cluster)
client

#!/usr/bin/env bash

#SBATCH -J dask-worker
#SBATCH -e /home/pleung/logs/slurm_logs/dask-worker-%J.err
#SBATCH -o /home/pleung/logs/slurm_logs/dask-worker-%J.out
#SBATCH -p medium
#SBATCH -n 1
#SBATCH --cpus-per-task=1
#SBATCH --mem=4G
#SBATCH -t 23:30:00

/home/pleung/.conda/envs/cereal/bin/python -m distributed.cli.dask_worker tcp://172.16.131.60:41777 --nthreads 1 --memory-limit 4.00GB --name name --nanny --death-timeout 600 --local-directory $TMPDIR/dask --no-nanny --no-dashboard



Client Scheduler: tcp://172.16.131.60:41777 Dashboard: http://172.16.131.60:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [7]:
# client.close(); cluster.close()

# Set command line options, make tasks and submit to client
previously used `-indexed_structure_store:fragment_store /home/bcov/sc/scaffold_comparison/data/ss_grouped_vall_all.h5`

In [7]:
import logging
import pyrosetta.distributed.io as io
from pyrosetta.distributed.cluster.core import PyRosettaCluster

logging.basicConfig(level=logging.INFO)
silents = glob(os.path.join(os.getcwd(), "02_silents/states.silent"))

options = { 
    "-out:level": "300",
    "-in:file:silent_struct_type": "binary",
    "-holes:dalphaball": "/home/bcov/ppi/tutorial_build/main/source/external/DAlpahBall/DAlphaBall.gcc",
    "-indexed_structure_store:fragment_store": "/net/databases/VALL_clustered/connect_chains/ss_grouped_vall_helix_shortLoop.h5",
    "-dunbrack_prob_buried": "0.8",
    "-dunbrack_prob_nonburied": "0.8", 
    "-dunbrack_prob_buried_semi": "0.8", 
    "-dunbrack_prob_nonburied_semi": "0.8",
}

def create_tasks(silents, options):
    for silent in silents:
        tasks = {"options": "-corrections::beta_nov16 true"}
        tasks["extra_options"] = options
        tasks["set_logging_handler"] = "interactive"
        tasks["-in:file:silent"] = silent
        yield tasks
        
if not os.getenv("DEBUG"):
    output_path = os.path.join(os.getcwd(), "03_enumerate_loops")
    PyRosettaCluster(
        tasks=create_tasks(silents, options),
        client=client,
        scratch_dir=output_path,
        output_path=output_path,
        ignore_errors=True,
    ).distribute(protocols=[load, loop_match, strict_remodel, label])

`conda env export --prefix /home/pleung/.conda/envs/cereal > environment.yml`
to reproduce this simulation later.


https://docs.anaconda.com/anaconda/install



INFO:pyrosetta.distributed:maybe_init performing pyrosetta initialization: {'options': '-run:constant_seed 1 -multithreading:total_threads 1', 'extra_options': '-mute all', 'set_logging_handler': 'interactive', 'silent': True}
INFO:pyrosetta.rosetta:Found rosetta database at: /home/pleung/.conda/envs/cereal/lib/python3.7/site-packages/pyrosetta/database; using it....
INFO:pyrosetta.rosetta:PyRosetta-4 2020 [Rosetta PyRosetta4.conda.linux.cxx11thread.serialization.CentOS.python37.Release 2020.50+release.1295438cd4bd2be39c9dbbfab8db669ab62415ab 2020-12-12T00:30:01] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.


WorkerError: 
Worker thread killed due to an error or segmentation fault encountered in the user-provided PyRosetta protocol '
Worker thread killed due to an error or segmentation fault encountered in the user-provided PyRosetta protocol 'strict_remodel'. '. 

# Look at scores
There is certainly a less embarrassing way to do this but at least this way is vectorized, so it should scale very well

In [11]:
def read_scorefile(scores):
    import pandas as pd
    scores = pd.read_json(scores, orient="records", typ="frame", lines=True)
    scores = scores.T
    mat = scores.values
    n = mat.shape[0]
    dicts = list(mat[range(n), range(n)])
    index = scores.index
    tabulated_scores = pd.DataFrame(dicts, index=index)
    return tabulated_scores
    
output_path = os.path.join(os.getcwd(), "03_enumerate_loops")
scores = os.path.join(output_path, "scores.json")
scores_df = read_scorefile(scores)
scores_df.head()

,abego_str,bb_clash,closure_type,ddg,dslf_fa13,dssp,fa_atr,fa_dun_dev,fa_dun_rot,fa_dun_semi,...,sc_int,scaffold,score,score_A,score_B,score_per_res,sfxn_used,shift,total_length,total_score
/mnt/home/pleung/projects/bistable_bundle/r3/hinges/03_enumerate_loops/decoys/0000/2021.02.10.18.46.42.089424_61c8955cc4b94ea1ad9b5a9aa8f74c27.pdb.bz2,XBAAAAAAAAAAAAAAAAAAAAAAAAAAAGBAAAAAAAAAAAAAAA...,85.404999,loop_match,-102.620003,0.0,LLHHHHHHHHHHHHHHHHHHHHHHHHHHHLLHHHHHHHHHHHHHHH...,-1380.576657,265.906119,176.846399,251.276903,...,0.743,DHR,0.0,-150.427994,-149.580994,-2.866,beta_nov16,-7.0,224.0,377.375224
/mnt/home/pleung/projects/bistable_bundle/r3/hinges/03_enumerate_loops/decoys/0000/2021.02.10.18.46.42.089424_29799d0d20f44d2e9b4bf0264e22f364.pdb.bz2,XAAAAAAAAAAAAAAAAAAAAAAAAABBGBAAAAAAAAAAAAAAAA...,68.002998,loop_match,-99.528000,0.0,LHHHHHHHHHHHHHHHHHHHHHHHHHLLLLHHHHHHHHHHHHHHHH...,-1475.992584,222.766275,208.715723,292.034501,...,0.778,DHR,0.0,-83.302002,-127.487000,-2.300,beta_nov16,-4.0,240.0,82.160442
/mnt/home/pleung/projects/bistable_bundle/r3/hinges/03_enumerate_loops/decoys/0000/2021.02.10.18.46.42.089424_f056f72f3bc44ce2859b1f51e83f53eb.pdb.bz2,XAAAAAAAAAAAAAAAAAAABABAAAAAAAAAAAAAAAAAAAAABB...,46.514999,loop_match,-84.594002,0.0,LHHHHHHHHHHHHHHHHHHHLLLHHHHHHHHHHHHHHHHHHHHHLL...,-1172.146208,129.714020,169.504620,188.562247,...,0.799,DHR,0.0,-126.384003,-140.666000,-2.849,beta_nov16,1.0,188.0,1171.656741
/mnt/home/pleung/projects/bistable_bundle/r3/hinges/03_enumerate_loops/decoys/0000/2021.02.10.18.46.42.089424_c750988750914f7fa6310c300d37f4a2.pdb.bz2,XAAAAAAAAAAAAAAAAAAGBBAAAAAAAAAAAAAAAAAGBBAAAA...,55.573002,loop_match,-62.223000,0.0,LHHHHHHHHHHHHHHHHHLLLLHHHHHHHHHHHHHHHHHLLLHHHH...,-1074.489270,157.579195,164.447718,222.339738,...,0.782,TH_DHR,0.0,-105.407997,-114.113998,-2.577,beta_nov16,7.0,172.0,1546.106509
/mnt/home/pleung/projects/bistable_bundle/r3/hinges/03_enumerate_loops/decoys/0000/2021.02.10.18.46.42.089424_60b3ef3bc50345f9ba1eb33b406bd30a.pdb.bz2,XBAAAAAAAAAAAAAAAAAAAAGBBAAAAAAAAAAAAAAAAAAAAG...,60.604000,loop_match,-88.192001,0.0,LLHHHHHHHHHHHHHHHHHHHLLLLHHHHHHHHHHHHHHHHHHHLL...,-1107.411732,197.054941,145.200228,165.424970,...,0.817,TH_DHR,0.0,-116.978996,-103.203003,-2.713,beta_nov16,-1.0,184.0,-121.628252


In [14]:
length = scores_df.query("closure_type == 'length_match'")

In [16]:
for i, row in length.iterrows():
    print(len(row["abego_str"]))
    print(row["parent_length"])

193
192.0
173
172.0
193
192.0
200
200.0


In [ ]:
scores_df.columns

In [ ]:
thrs = scores_df[scores_df["scaffold"].str.contains("THR")]
dhrs = scores_df[scores_df["scaffold"].str.match("DHR")]
th_dhrs = scores_df[scores_df["scaffold"].str.match("TH_DHR")]

In [ ]:
sns.set(
    context="talk",
    font_scale=2, # make the font larger; default is pretty small
    style="ticks", # make the background white with black lines
    palette="colorblind" # a color palette that is colorblind friendly!
)

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(ncols=2, nrows=2, figsize=(30, 20), tight_layout=False)
scores_df.groupby("closure_type").size().plot(kind="pie", autopct="%1.2f%%", ax=ax1)
thrs.groupby("closure_type").size().plot(kind="pie", autopct="%1.2f%%", ax=ax2)
dhrs.groupby("closure_type").size().plot(kind="pie", autopct="%1.2f%%", ax=ax3)
th_dhrs.groupby("closure_type").size().plot(kind="pie", autopct="%1.2f%%", ax=ax4)
ax1.set_ylabel("All")
ax2.set_ylabel("THRs")
ax3.set_ylabel("DHRs")
ax4.set_ylabel("TH_DHRs")
plt.savefig("figs/03_looping_counts_by_scaffold.png")

In [ ]:
scores_df["pivot_helix"] = scores_df["pivot_helix"].astype(float).astype(int)
scores_df["pre_break_helix"] = scores_df["pre_break_helix"].astype(float).astype(int)
scores_df["shift"] = scores_df["shift"].astype(float).astype(int)

In [ ]:
# from matplotlib import ticker
# from matplotlib.ticker import MultipleLocator, FormatStrFormatter
# sns.set(
#     context="talk",
#     font_scale=2, # make the font larger; default is pretty small
#     style="ticks", # make the background white with black lines
#     palette="colorblind" # a color palette that is colorblind friendly!
# )

# order = list(set(scores_df["shift"].values))
# order.sort()

# fig = plt.figure(figsize=(20,10), tight_layout=True)
# pivot_order = [4, 5, 6, 7]
# ax = sns.boxplot(
#     x="shift", y="score_per_res", data=scores_df,
#     order= order, showfliers=False,
#     hue="pivot_helix", hue_order=pivot_order,
# )
# ax.legend(loc="upper right")
# ax.set_ylim(-5, 0)
# sns.despine()
# plt.setp(ax.get_legend().get_title(), fontsize='24')
# plt.xticks(rotation=90)
# plt.savefig("figs/02_shift_vs_score_per_res_groupby_pivot.png")

# Parallel unzipping, relabeling and packing into a silent of all selected designs
Method maintains score info

In [ ]:
import pyrosetta.distributed.io as io
from pyrosetta.distributed.packed_pose.core import PackedPose

def unpack_add_scores_repack(bz2file:str, scores:dict) -> PackedPose:
    import bz2
    import os
    import pyrosetta
    import pyrosetta.distributed.io as io
    with open(bz2file, "rb") as f:
        ppose = io.pose_from_pdbstring(bz2.decompress(f.read()).decode())
    pose = io.to_pose(ppose)
    basename =  os.path.basename(bz2file.replace(".pdb.bz", "", 1))
    pose.pdb_info().name(basename)
    for key, value in scores.items():
        pyrosetta.rosetta.core.pose.setPoseExtraScore(pose, key, value)
    final_ppose = io.to_packed(pose)
    return final_ppose

if not os.getenv("DEBUG"):
    future_pposes = []
    for i, bz2file in enumerate(scores_df.index, start=1): # TODO
        # decompress from bz2, add score info and pack
        scores = dict(scores_df.loc[bz2file])
        if scores["closure_type"] != "not_closed":
            future_ppose = client.submit(unpack_add_scores_repack, bz2file, scores)
            future_pposes.append(future_ppose)
    pposes = []
    for future in tqdm(future_pposes):
        pposes.append(future.result())

out_path = os.path.join(os.getcwd(), "03_silents")
silent_name = "closed.silent"

os.makedirs(out_path, exist_ok=True)
io.to_silent(pposes, os.path.join(out_path, silent_name))
msg = """
out_path: {out_path}
packed {i} poses
""".format(out_path=out_path, i=i)
print(msg)

In [ ]:
client.close(); cluster.close()